The task is to develop and evaluate a two-stage information retrieval model
that given a query returns the n most relevant documents and then ranks the sentences within
the documents.

In [1]:
import os
cwd = os.getcwd()
print(cwd)
os.chdir("C:/Users/nobel/OneDrive/Desktop/FinalProject_7002401_7002402/FinalProject_7002401_7002402/FinalProject_7002401_7002402")

C:\Users\nobel


In [2]:
#import necessary packages

import nltk
import numpy as np
from bs4 import BeautifulSoup
import operator
import itertools
import gensim
import string
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

#reading file

infile = open("new_1.xml","r")
contents = infile.read()
soup = BeautifulSoup(contents,'lxml')

Extract the text and document numbers from the corpus

In [3]:
#store document IDs

query_sentence_rank={}
document_numbers=[]
for doc in soup.find_all('doc'):
    temp=doc.docno.text
    document_numbers.append(temp)
    
print(document_numbers)

['FT943-15865', 'FT943-15885', 'FT943-15933', 'FT943-15981', 'FT943-16103', 'FT943-16115', 'FT943-16120', 'FT943-16128', 'FT943-16220', 'FT943-16263', 'FT943-16280', 'FT943-16300', 'FT943-16397', 'FT943-16408', 'FT943-16421', 'FT943-16449', 'FT943-16602', 'FT943-16632', 'FT943-16675', 'FT943-16710', 'FT943-16749', 'FT943-16852', 'FT943-16874', 'FT943-16996', 'FT943-17008', 'FT943-17010', 'FT943-17019', 'FT943-17054', 'FT943-17109', 'FT943-17115', 'FT943-1541', 'FT943-1550', 'FT943-1619', 'FT943-1701', 'FT943-1802', 'FT943-1829', 'FT943-1831', 'FT943-1833', 'FT943-1914', 'FT943-1933', 'FT943-2065', 'FT943-2094', 'FT943-2131', 'FT943-2175', 'FT943-2207', 'FT943-2211', 'FT943-2281', 'FT943-2340', 'FT943-2450', 'FT943-2479', 'FT943-2630', 'FT943-2672', 'FT943-2776', 'FT943-2912', 'FT943-2971', 'FT943-3089', 'FT944-4', 'FT944-17', 'FT944-23', 'FT944-56', 'FT944-143', 'FT944-3005', 'FT944-3008', 'FT944-3015', 'FT944-3048', 'FT944-3061', 'FT944-3089', 'FT944-3139', 'FT944-3192', 'FT944-3210',

In [4]:
#read contents of <text> tag and store it
#pre-processing the data

document_text=[]
var1=[]
for doc in soup.find_all('text'):
    temp2=doc.text
    precision_text=doc.text
    precision_text=' '.join(precision_text.split())
    var1.append(precision_text)
    temp2=temp2.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    temp2 = tokenizer.tokenize(temp2)
    document_text.append(temp2)
    
print(document_text[0]) 

['peter', 'phillipson', 'marketing', 'director', 'international', 'region', 'for', 'united', 'distillers', 'has', 'been', 'appointed', 'commercial', 'director', 'of', 'owners', 'abroad', 'tony', 'coleman', 'joins', 'as', 'group', 'human', 'resources', 'director', 'from', 'grand', 'met']


In [5]:
#storing document number and text as a dictionary

res = dict(zip(document_numbers, document_text))

new_data = dict(zip(document_numbers, var1))

print(res['FT943-15865'])

['peter', 'phillipson', 'marketing', 'director', 'international', 'region', 'for', 'united', 'distillers', 'has', 'been', 'appointed', 'commercial', 'director', 'of', 'owners', 'abroad', 'tony', 'coleman', 'joins', 'as', 'group', 'human', 'resources', 'director', 'from', 'grand', 'met']




Inverse document frequency (idf) of the terms idf(term(i)) = log(N)/n(term(i))

where N is the total number of documents and n(term(i)) is the number of documents that
contain the term(i)


In [6]:
number_of_documents_containing_words= {}

for key in res:
    words = res[key]
    for i in words:
        #needs to be unique
        try:
            number_of_documents_containing_words[i].add(key)
        except:
            number_of_documents_containing_words[i] = {key}
            


for i in number_of_documents_containing_words:
    number_of_documents_containing_words[i]=len(number_of_documents_containing_words[i])
    
#total number of documents
total_documents=len(res)

number_of_documents_containing_words

{'peter': 6,
 'phillipson': 1,
 'marketing': 3,
 'director': 27,
 'international': 16,
 'region': 2,
 'for': 104,
 'united': 3,
 'distillers': 1,
 'has': 80,
 'been': 57,
 'appointed': 12,
 'commercial': 7,
 'of': 141,
 'owners': 4,
 'abroad': 4,
 'tony': 1,
 'coleman': 1,
 'joins': 2,
 'as': 58,
 'group': 32,
 'human': 4,
 'resources': 4,
 'from': 70,
 'grand': 2,
 'met': 4,
 'cumbria': 2,
 'motor': 3,
 'services': 4,
 'which': 61,
 'operates': 3,
 'the': 143,
 'majority': 8,
 'bus': 1,
 'in': 119,
 'county': 1,
 'banned': 3,
 'cigarette': 1,
 'and': 116,
 'alcohol': 2,
 'advertising': 3,
 'its': 57,
 'buses': 2,
 'rights': 3,
 'workers': 6,
 'around': 9,
 'world': 20,
 'are': 47,
 'falling': 6,
 'victim': 1,
 'to': 120,
 'kind': 4,
 'torture': 1,
 'ill': 3,
 'treatment': 6,
 'other': 33,
 'violations': 1,
 'they': 28,
 'trying': 6,
 'prevent': 9,
 'amnesty': 1,
 'says': 21,
 'today': 15,
 'annual': 9,
 'report': 5,
 '1993': 7,
 'reuter': 4,
 'reports': 8,
 'london': 16,
 'police': 4,

In [7]:
#computing inverse document frequency(IDF) of the terms

idf_terms={}
for key in number_of_documents_containing_words:
    idf_terms[key]=np.log((total_documents)/(number_of_documents_containing_words[key]))
    
print(idf_terms)

{'peter': 3.3440389678222067, 'phillipson': 5.135798437050262, 'marketing': 4.037186148382152, 'director': 1.8399615710459327, 'international': 2.3632097148104805, 'region': 4.442651256490317, 'for': 0.4914075379088891, 'united': 4.037186148382152, 'distillers': 5.135798437050262, 'has': 0.7537718023763802, 'been': 1.0927471692157116, 'appointed': 2.6508917872622613, 'commercial': 3.1898882879949486, 'of': 0.18703854667209346, 'owners': 3.7495040759303713, 'abroad': 3.7495040759303713, 'tony': 5.135798437050262, 'coleman': 5.135798437050262, 'joins': 4.442651256490317, 'as': 1.0753554265038423, 'group': 1.6700625342505353, 'human': 3.7495040759303713, 'resources': 3.7495040759303713, 'from': 0.8873031950009027, 'grand': 4.442651256490317, 'met': 3.7495040759303713, 'cumbria': 4.442651256490317, 'motor': 4.037186148382152, 'services': 3.7495040759303713, 'which': 1.0249245728769505, 'operates': 4.037186148382152, 'the': 0.17295380679035444, 'majority': 3.056356895370426, 'bus': 5.135798



Compute and store the term frequency (tf) of the terms per document along with the
document ID:
tf(term(i), doc) = n(termi, doc)/max(termj , doc)

where n(termi, doc) is the number of times term(i) appears in the document and max(term(j) , doc)
is the number of times the most frequent term of the document appears in the document.

In [8]:
#computing term frequency(TF) of the terms per document

def calculate_tf_per_document(word_list):
    Tf_scores={}
    max_element=0
    for each_word in word_list:
        if each_word in Tf_scores:
            Tf_scores[each_word] += 1
        else:
            Tf_scores[each_word] = 1
    for each_word in Tf_scores:
        if(Tf_scores[each_word]>max_element):
            max_element=Tf_scores[each_word]
    for each_word in Tf_scores:
        Tf_scores[each_word]=Tf_scores[each_word]/max_element
    return(Tf_scores)


In [9]:
resulting_dictionary_document_number_tf_values={}

for every_document in res:
    dictionary_with_tf_scores_of_each_document=calculate_tf_per_document(res[every_document])
    resulting_dictionary_document_number_tf_values[every_document]=dictionary_with_tf_scores_of_each_document
    
    
resulting_dictionary_document_number_tf_values   

{'FT943-15865': {'peter': 0.3333333333333333,
  'phillipson': 0.3333333333333333,
  'marketing': 0.3333333333333333,
  'director': 1.0,
  'international': 0.3333333333333333,
  'region': 0.3333333333333333,
  'for': 0.3333333333333333,
  'united': 0.3333333333333333,
  'distillers': 0.3333333333333333,
  'has': 0.3333333333333333,
  'been': 0.3333333333333333,
  'appointed': 0.3333333333333333,
  'commercial': 0.3333333333333333,
  'of': 0.3333333333333333,
  'owners': 0.3333333333333333,
  'abroad': 0.3333333333333333,
  'tony': 0.3333333333333333,
  'coleman': 0.3333333333333333,
  'joins': 0.3333333333333333,
  'as': 0.3333333333333333,
  'group': 0.3333333333333333,
  'human': 0.3333333333333333,
  'resources': 0.3333333333333333,
  'from': 0.3333333333333333,
  'grand': 0.3333333333333333,
  'met': 0.3333333333333333},
 'FT943-15885': {'cumbria': 0.5,
  'motor': 0.5,
  'services': 1.0,
  'which': 0.5,
  'operates': 0.5,
  'the': 1.0,
  'majority': 0.5,
  'of': 0.5,
  'bus': 0.5,
 

 Given a list of query terms, you can compute and store the tf-idf weights for these terms
as the product of the term’s idf and the tf-value of the term in the respective document: tf − idf(termi, doc) = tf(term, doc) ∗ idf(termi) 

In [10]:
#computing and storing tf-idf weights

def calculate_tf_idf_per_document(tf_list):
    TF_IDF={}
    count=0
    for each_word in tf_list:
                TF_IDF[each_word]=tf_list[each_word]*idf_terms[each_word]
    return(TF_IDF)


In [11]:
resulting_dictionary_document_number_tf_idf_values={}


for every_document in resulting_dictionary_document_number_tf_values:
    dictionary_with_tf_idf_scores_of_each_document=calculate_tf_idf_per_document(resulting_dictionary_document_number_tf_values[every_document])
    resulting_dictionary_document_number_tf_idf_values[every_document]=dictionary_with_tf_idf_scores_of_each_document


Based on the tf-idf weights, you can determine the relevance of the documents for the query
based on their cosine similarity with the query using dot products. To calculate the tf-idf
weights for the query, you should treat it like a document.
cos(q ,d ) = q * d / |q|* |d |

In [12]:
#read questions or query file

infile2 = open("test_questions.txt","r")
contents2 = infile2.read()
soup2 = BeautifulSoup(contents2,'lxml')

In [13]:
#read contents of <desc> tag and store it
#pre-processing the data

query_text=[]
for descriptions in soup2.find_all('desc'):
    temp_description=descriptions.text
    temp_description=temp_description.replace('Description:','')
    temp_description=temp_description.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    temp_description = tokenizer.tokenize(temp_description)
    query_text.append(temp_description)

In [14]:
#store query numbers

query_numbers=[]
for i in range(1,101):
    query_numbers.append(i)

In [15]:
#storing query number and text as a dictionary

query_final = dict(zip(query_numbers, query_text))


In [16]:
resulting_query_number_tf_values={}

for every_query in query_final:
    dictionary_with_tf_scores_of_each_query=calculate_tf_per_document(query_final[every_query])
    resulting_query_number_tf_values[every_query]=dictionary_with_tf_scores_of_each_query



In [17]:
#compute tf-idf for query file
def calculate_tf_idf_per_query(tf_list):
    TF_IDF={}
    count=0
    for each_word in tf_list:
        if each_word in idf_terms:
                TF_IDF[each_word]=tf_list[each_word]*idf_terms[each_word]
        else:
            TF_IDF[each_word]=0
    return(TF_IDF)





In [18]:
resulting_dictionary_query_number_tf_idf_values={}

for every_query in resulting_query_number_tf_values:
    dictionary_with_tf_idf_scores_of_each_query=calculate_tf_idf_per_query(resulting_query_number_tf_values[every_query])
    resulting_dictionary_query_number_tf_idf_values[every_query]=dictionary_with_tf_idf_scores_of_each_query




In [19]:
#vectorizing

def vectorize(every_document):
    D = np.zeros(len(idf_terms))
    for i, word in enumerate(idf_terms):
        if word in every_document:
                D[i] = every_document[word]
    return(D)              


#computing cosine similarity scores

def calculate_cosine_similarity(document_key,query_number):
    a=vectorize(resulting_dictionary_document_number_tf_idf_values[document_key])
    b=vectorize(resulting_dictionary_query_number_tf_idf_values[query_number])
    cosine_similarity_score=np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cosine_similarity_score
  
    

In [20]:
#input query number from user

query_number= int(input("enter query number ")) 

resulting_query_cosine_scores={}

for key in resulting_dictionary_document_number_tf_idf_values:
    dictionary_with_cosine_scores_of_each_query_with_document=calculate_cosine_similarity(key,query_number)
    resulting_query_cosine_scores[key]=dictionary_with_cosine_scores_of_each_query_with_document

    

enter query number 1


In [21]:
#display top 50 sorted cosine similarity scores along with document ID

sorted_d = dict(sorted(resulting_query_cosine_scores.items(), key=operator.itemgetter(1),reverse=True))
ut = dict(itertools.islice(sorted_d.items(), 50))
print(ut)

{'FT944-3005': 0.15018716144223968, 'FT944-5914': 0.04722393520423186, 'FT943-16280': 0.03896351744666071, 'FT944-4736': 0.03256033656582329, 'FT943-2065': 0.031178547285723866, 'FT944-3987': 0.02613307827560824, 'FT944-5256': 0.026095774579396915, 'FT943-2971': 0.02586895101070965, 'FT944-5263': 0.022747634448643382, 'FT944-3898': 0.021525982462305155, 'FT944-3783': 0.021010120421529308, 'FT944-574': 0.020241640145227426, 'FT944-4482': 0.02009561704104529, 'FT944-4047': 0.019469001265957624, 'FT944-6815': 0.01905894134554807, 'FT944-3513': 0.018244826087345192, 'FT944-3139': 0.016230566350003084, 'FT943-2672': 0.015720840529084335, 'FT944-3740': 0.01485641995065856, 'FT944-419': 0.014238862717772267, 'FT943-16421': 0.013721767144253782, 'FT944-3521': 0.012690261851331637, 'FT944-3015': 0.011655225643433723, 'FT943-1550': 0.011539964523542001, 'FT944-552': 0.011390849470548683, 'FT943-2175': 0.011127482603922682, 'FT944-441': 0.010812083686593348, 'FT944-6608': 0.010805185040515511, 'F

In [22]:
#computing precision values

import re
intersect = []
for item in ut:
    if item in new_data:
        intersect.append(new_data[item])

#input relevant regex pattern

test_list_cosine_scores = [] 
n = int(input("Enter number of  regex elements : ")) 
for i in range(0, n): 
    ele = input()
  
    test_list_cosine_scores.append(ele)
    
    
temp_cosine = '(?:% s)' % '|'.join(test_list_cosine_scores) 


#initialize count varaible

ct_cosine=0

for index,value in enumerate(intersect):
    if re.search(temp_cosine,intersect[index]):
        ct_cosine=ct_cosine+1
print("The total relevant count is ",ct_cosine)

#precision at r

r=50
given_precision_cosine=ct_cosine/r

print("The precision value is ",given_precision_cosine)

Enter number of  regex elements : 1
Margaret
The total relevant count is  1
The precision value is  0.02


Use the baseline model from Section 1 and return the top 1000 documents given a query.

Re-rank the top 1000 documents with a more advanced approach like BM253. You should return the top 50 documents based on the top 1000 documents returned in (a).

In [24]:
#Advanced Document Retriever with re-ranking
#Implementing question using bm25

sorted_d1 = dict(sorted(resulting_query_cosine_scores.items(), key=operator.itemgetter(1),reverse=True))
top_1000_documents=dict(itertools.islice(sorted_d1.items(), 1000))
print("The top 1000 documents scores based on baseline model are:\n",top_1000_documents,"\n\n\n")

document_numbers_top_1000_documents=[]

for key in top_1000_documents:
    document_numbers_top_1000_documents.append(key)
    


corpus_to_be_intitialized=[]

for key in top_1000_documents:
    corpus_to_be_intitialized.append(res[key])

bm25=gensim.summarization.bm25.BM25(corpus_to_be_intitialized)
scores=bm25.get_scores(query_final[query_number])    
#print(scores)

dictionary_bm_25_scores = dict(zip(document_numbers_top_1000_documents, scores))
    
sorted__bm_25 = dict(sorted(dictionary_bm_25_scores.items(), key=operator.itemgetter(1),reverse=True))
bt = dict(itertools.islice(sorted__bm_25.items(), 50))
print("The bm25 score values are..\n",bt)


The top 1000 documents scores based on baseline model are:
 {'FT944-3005': 0.15018716144223968, 'FT944-5914': 0.04722393520423186, 'FT943-16280': 0.03896351744666071, 'FT944-4736': 0.03256033656582329, 'FT943-2065': 0.031178547285723866, 'FT944-3987': 0.02613307827560824, 'FT944-5256': 0.026095774579396915, 'FT943-2971': 0.02586895101070965, 'FT944-5263': 0.022747634448643382, 'FT944-3898': 0.021525982462305155, 'FT944-3783': 0.021010120421529308, 'FT944-574': 0.020241640145227426, 'FT944-4482': 0.02009561704104529, 'FT944-4047': 0.019469001265957624, 'FT944-6815': 0.01905894134554807, 'FT944-3513': 0.018244826087345192, 'FT944-3139': 0.016230566350003084, 'FT943-2672': 0.015720840529084335, 'FT944-3740': 0.01485641995065856, 'FT944-419': 0.014238862717772267, 'FT943-16421': 0.013721767144253782, 'FT944-3521': 0.012690261851331637, 'FT944-3015': 0.011655225643433723, 'FT943-1550': 0.011539964523542001, 'FT944-552': 0.011390849470548683, 'FT943-2175': 0.011127482603922682, 'FT944-441': 

In [25]:
#Sentence Ranker
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

query_text_sentence=[]
for descriptions in soup2.find_all('desc'):
    temp_description_sentence=descriptions.text
    temp_description_sentence=temp_description_sentence.replace('Description:','')
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    temp_description_sentence = tokenizer.tokenize(temp_description_sentence)
    query_text_sentence.append(temp_description_sentence)


query_final_sentence = dict(zip(query_numbers, query_text_sentence))


document_text_sentence_tokenize=[]
for doc in soup.find_all('text'):
    temp_sentence_tokenize_2=doc.text 
    temp_sentence_tokenize_2=' '.join(temp_sentence_tokenize_2.split())
    temp_sentence_tokenize_2 = sent_tokenize(temp_sentence_tokenize_2)
    document_text_sentence_tokenize.append(temp_sentence_tokenize_2)    

res_sentence = dict(zip(document_numbers, document_text_sentence_tokenize))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nobel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
corpus_to_be_initialized_sentence=[]


for key in bt:
    #print(res_sentence[key])
    tokenized_sents = [word_tokenize(i) for i in res_sentence[key]]
    corpus_to_be_initialized_sentence.extend(tokenized_sents)

#print(corpus_to_be_initialized_sentence)
    
for key,value in enumerate(corpus_to_be_initialized_sentence):
    corpus_to_be_initialized_sentence[key]=[''.join(c for c in s if c not in string.punctuation) for s in corpus_to_be_initialized_sentence[key]]
    corpus_to_be_initialized_sentence[key] = [s for s in corpus_to_be_initialized_sentence[key] if s]

In [27]:
bm25=gensim.summarization.bm25.BM25(corpus_to_be_initialized_sentence)
scores_sentence=bm25.get_scores(query_final_sentence[query_number])    


dictionary_sentence_bm_25_scores=dict(zip(scores_sentence, corpus_to_be_initialized_sentence))



sorted_list=[]
for key in sorted(dictionary_sentence_bm_25_scores.keys()):
    sorted_list.append(dictionary_sentence_bm_25_scores[key])

   
    
reversed_list_dictionary_sentence_bm_25_scores = sorted_list[::-1] 

sliced_list_dictionary_sentence_bm_25_scores=reversed_list_dictionary_sentence_bm_25_scores[:50]

print("The top 50 sentences retrieved based on bm25 scores are:\n",sliced_list_dictionary_sentence_bm_25_scores) 

The top 50 sentences retrieved based on bm25 scores are:
 [['The', 'author', 'is', 'a', 'London', 'general', 'practitioner'], ['Who', 'was', 'the', 'baroness', 'and', 'whose', 'were', 'the', 'bottoms'], ['2', 'Who', 'redealt', 'the', 'cards', 'to', 'play', 'the', 'king', 'on', 'television'], ['20', 'Who', 'left', 'the', 'boulevard', 'under', 'a', 'cloud', 'without', 'enjoying', 'the', 'sunset'], ['Mr', 'Graham', 'Sharpe', 'who', 'runs', 'a', 'book', 'on', 'the', 'Christmas', 'number', 'one', 'for', 'William', 'Hill', 'the', 'bookmaker', 'said', 'This', 'is', 'one', 'of', 'the', 'most', 'exciting', 'Christmas', 'charts', 'ever'], ['19', 'Who', 'left', 'one', 'Nirvana', 'in', 'the', 'hope', 'of', 'finding', 'another'], ['To', 'see', 'Charles', 'Dickens', 's', 'A', 'Christmas', 'Carol', 'onstage', 'is', 'to', 'know', 'again', 'with', 'a', 'frisson', 'of', 'joy', 'how', 'theatrical', 'its', 'author', 's', 'instincts', 'were'], ['Brian', 'Hardy', 'director', 'of', 'finance', 'at', 'Burmah',